<a href="https://colab.research.google.com/github/XiaoTanOvO/-/blob/main/%E9%87%91%E8%9E%8D%E5%B8%82%E5%A0%B4%E6%A6%82%E8%BF%B0%E8%88%87%E8%B3%87%E6%96%99%E6%94%B6%E9%9B%86%E5%8F%8A%E9%A0%90%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 yfinance matplotlib

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

# 定義爬取資料的函式
def get_twse_stock_data(stock_id, date):
    url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={date}&stockNo={stock_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 解析表格資料
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')
    data = []

    for row in rows[2:]:  # 跳過前兩行標題
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if len(cols) > 0:
            data.append(cols)

    # 建立 DataFrame
    columns = ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    df = pd.DataFrame(data, columns=columns)

    # 轉換數據類型
    df['日期'] = df['日期'].str.replace(',', '')
    df['成交股數'] = df['成交股數'].str.replace(',', '').astype(int)
    df['成交金額'] = df['成交金額'].str.replace(',', '').astype(int)
    df['開盤價'] = df['開盤價'].str.replace(',', '').astype(float)
    df['最高價'] = df['最高價'].str.replace(',', '').astype(float)
    df['最低價'] = df['最低價'].str.replace(',', '').astype(float)
    df['收盤價'] = df['收盤價'].str.replace(',', '').astype(float)
    df['漲跌價差'] = df['漲跌價差'].str.replace(',', '')
    df['成交筆數'] = df['成交筆數'].str.replace(',', '').astype(int)

    return df

# 範例使用
stock_id = '2330'  # 台積電
date = '20230901'  # 查詢日期範例
twse_df = get_twse_stock_data(stock_id, date)
print(twse_df.head())

# 可視化
# plt.figure(figsize=(14, 7))
# plt.plot(twse_df['日期'], twse_df['收盤價'], marker='o', linestyle='-')
# plt.title(f'TWSE Stock Price for {stock_id}')
# plt.xlabel('Date')
# plt.ylabel('Closing Price')
# plt.grid(True)
# plt.show()


          日期      成交股數         成交金額    開盤價    最高價    最低價    收盤價   漲跌價差   成交筆數
0  112/09/01  15194921   8331995536  543.0  553.0  543.0  548.0  -1.00  12739
1  112/09/04   9772925   5412329762  549.0  557.0  549.0  557.0  +9.00  12158
2  112/09/05  14337041   7911531475  553.0  555.0  550.0  552.0  -5.00  13714
3  112/09/06  14442757   7972706415  556.0  556.0  550.0  550.0  -2.00  12752
4  112/09/07  22610385  12305542681  546.0  548.0  542.0  542.0  -8.00  41564


In [10]:
import yfinance as yf

# 使用 yfinance 獲取 Yahoo 股價資料
def get_yahoo_stock_data(ticker):
    stock = yf.Ticker(ticker)
    df = stock.history(period="1mo")
    return df

# 範例使用
ticker = "2330.TW"  # 台積電在 Yahoo 的代號
yahoo_df = get_yahoo_stock_data(ticker)
print(yahoo_df.head())

# 可視化
# plt.figure(figsize=(14, 7))
# plt.plot(yahoo_df.index, yahoo_df['Close'], marker='o', linestyle='-')
# plt.title(f'Yahoo Stock Price for {ticker}')
# plt.xlabel('Date')
# plt.ylabel('Closing Price')
# plt.grid(True)
# plt.show()


                                 Open        High         Low       Close  \
Date                                                                        
2024-08-20 00:00:00+08:00  972.662478  972.662478  963.702435  968.680237   
2024-08-21 00:00:00+08:00  956.733507  959.720188  951.755705  953.746826   
2024-08-22 00:00:00+08:00  953.746816  959.720178  943.791212  946.777893   
2024-08-23 00:00:00+08:00  939.808942  947.773424  934.831140  944.786743   
2024-08-26 00:00:00+08:00  952.751271  956.733513  940.804547  945.782349   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-08-20 00:00:00+08:00  24039959        0.0           0.0  
2024-08-21 00:00:00+08:00  29758188        0.0           0.0  
2024-08-22 00:00:00+08:00  24349754        0.0           0.0  
2024-08-23 00:00:00+08:00  27592153        0.0           0.0  
2024-08-26 00:00:00+08:00  25057581        0.0           0.0  
